In [ ]:
!pip install SimpleITK

In [2]:
import os
import pandas as pd
import SimpleITK as sitk
import numpy as np
import pydicom
import nibabel as nib
from tqdm import tqdm
import matplotlib.pyplot as plt
import shutil

In [3]:
# 1. 전체 프로젝트 경로
BASE_DIR = r'C:\conda\3. Project\rsna-2022-cervical-spine-fracture-detection'
# 2. 실제 데이터가 들어있는 old_train 경로
DATA_DIR = os.path.join(BASE_DIR, 'old_train')

# --- 입력 경로 ---
TRAIN_CSV = os.path.join(DATA_DIR, 'train.csv')
BBOX_CSV = os.path.join(DATA_DIR, 'train_bounding_boxes.csv')
IMAGE_DIR = os.path.join(DATA_DIR, 'train_images')    # old_train 폴더 안의 이미지
SEG_DIR = os.path.join(DATA_DIR, 'segmentations')     # old_train 폴더 안의 마스크

# --- 출력/처리 경로 ---
# 변환된 NIfTI 이미지를 저장할 곳 (old_train/converted_images)
CONV_DIR = os.path.join(DATA_DIR, 'converted_images') 
PROC_DIR = os.path.join(DATA_DIR, 'processed_masks')
SUMMARY_DIR = os.path.join(DATA_DIR, 'patient_summaries') 
GOLDEN_DATA_DIR = os.path.join(BASE_DIR, 'golden_dataset')
REPORT_DIR = os.path.join(DATA_DIR, 'alignment_reports') # 결과 저장 폴더

# 필터링된 결과물
FILTERED_TRAIN_CSV = os.path.join(GOLDEN_DATA_DIR, 'filtered_train.csv')
FILTERED_BBOX_CSV = os.path.join(GOLDEN_DATA_DIR, 'filtered_train_bounding_boxes.csv')

# 데이터프레임 로드 (필요시)
# train_df = pd.read_csv(TRAIN_CSV)
# df_bbox = pd.read_csv(BBOX_CSV)

print(f"설정된 이미지 경로: {IMAGE_DIR}")
print(f"설정된 변환 저장 경로: {CONV_DIR}")


설정된 이미지 경로: C:\conda\3. Project\rsna-2022-cervical-spine-fracture-detection\old_train\train_images
설정된 변환 저장 경로: C:\conda\3. Project\rsna-2022-cervical-spine-fracture-detection\old_train\converted_images


## Dicom -> Nifti 변환 (mask 유 환자들 대상)

In [4]:
def convert_specific_patients(seg_dir, train_img_dir, conv_dir):
    """
    세그멘테이션 파일이 있는 환자만 골라서 DICOM 시리즈를 NIfTI 파일로 변환합니다.
    """
    # 1. 저장할 새 폴더 생성
    if not os.path.exists(conv_dir):
        os.makedirs(conv_dir)
        print(f"✅ 폴더 생성됨: {conv_dir}")

    # 2. 세그멘테이션 폴더 내 파일 목록 가져오기
    seg_files = [f for f in os.listdir(seg_dir) if f.endswith('.nii') or f.endswith('.nii.gz')]
    
    print(f"🔍 총 {len(seg_files)}명의 마스크 보유 환자를 찾았습니다. 변환을 시작합니다.")

    for seg_file in seg_files:
        # 3. 파일명에서 환자 UID 추출 (.nii.gz 또는 .nii 대응)
        patient_uid = seg_file.split('.nii')[0]
        
        # 4. 해당 환자의 DICOM 폴더 및 저장 경로 설정
        dicom_dir = os.path.join(train_img_dir, patient_uid)
        output_path = os.path.join(conv_dir, f"{patient_uid}_image.nii.gz")

        # 5. 이미 변환된 파일이 있는지 체크 (중복 작업 방지)
        if os.path.exists(output_path):
            # print(f"⏭️ 이미 존재함: {patient_uid}")
            continue

        # 6. DICOM 폴더 확인 후 변환
        if os.path.exists(dicom_dir):
            try:
                print(f"📂 [{patient_uid}] 변환 중...")
                
                # DICOM 읽기
                reader = sitk.ImageSeriesReader()
                dicom_names = reader.GetGDCMSeriesFileNames(dicom_dir)
                
                if not dicom_names:
                    print(f"⚠️ {patient_uid}: DICOM 파일이 없습니다.")
                    continue
                    
                reader.SetFileNames(dicom_names)
                image = reader.Execute()
                
                # 방향 교정 (마스크와 축을 맞추기 위해 LAS 고정)
                # SimpleITK 버전에 따라 DICOMOrient 또는 OrientImageFilter 사용
                image = sitk.DICOMOrient(image, 'LAS')
                
                # NIfTI 저장
                sitk.WriteImage(image, output_path)
                
            except Exception as e:
                print(f"❌ {patient_uid} 변환 실패: {e}")
        else:
            print(f"⚠️ 경고: 마스크는 있으나 DICOM 폴더가 없습니다: {dicom_dir}")

    print("\n✨ 모든 작업이 완료되었습니다!")

# 실행 코드 (필요할 때 주석 해제)
convert_specific_patients(SEG_DIR, IMAGE_DIR, CONV_DIR)

✅ 폴더 생성됨: C:\conda\3. Project\rsna-2022-cervical-spine-fracture-detection\old_train\converted_images
🔍 총 87명의 마스크 보유 환자를 찾았습니다. 변환을 시작합니다.
📂 [1.2.826.0.1.3680043.10633] 변환 중...
📂 [1.2.826.0.1.3680043.10921] 변환 중...
📂 [1.2.826.0.1.3680043.11827] 변환 중...
📂 [1.2.826.0.1.3680043.11988] 변환 중...
📂 [1.2.826.0.1.3680043.12281] 변환 중...
📂 [1.2.826.0.1.3680043.12292] 변환 중...
📂 [1.2.826.0.1.3680043.12833] 변환 중...
📂 [1.2.826.0.1.3680043.1363] 변환 중...
📂 [1.2.826.0.1.3680043.14267] 변환 중...
📂 [1.2.826.0.1.3680043.1480] 변환 중...
📂 [1.2.826.0.1.3680043.15206] 변환 중...
📂 [1.2.826.0.1.3680043.1542] 변환 중...
📂 [1.2.826.0.1.3680043.1573] 변환 중...
📂 [1.2.826.0.1.3680043.16092] 변환 중...
📂 [1.2.826.0.1.3680043.16919] 변환 중...
📂 [1.2.826.0.1.3680043.17481] 변환 중...
📂 [1.2.826.0.1.3680043.17960] 변환 중...
📂 [1.2.826.0.1.3680043.18480] 변환 중...
📂 [1.2.826.0.1.3680043.1868] 변환 중...
📂 [1.2.826.0.1.3680043.18906] 변환 중...
📂 [1.2.826.0.1.3680043.18935] 변환 중...
📂 [1.2.826.0.1.3680043.18968] 변환 중...
📂 [1.2.826.0.1.3680043.19021] 

## nifti -> npy 슬라이스 (mask 유 환자들 대상)

In [7]:
# 2. 마스크 저장 폴더 생성
if not os.path.exists(PROC_DIR):
    os.makedirs(PROC_DIR)
    print(f"✅ 폴더 생성됨: {PROC_DIR}")

def process_and_align_fixed(seg_dir, img_dir, output_dir):
    # 세그멘테이션 파일 목록 (87명 분량)
    seg_files = [f for f in os.listdir(seg_dir) if f.endswith('.nii.gz') or f.endswith('.nii')]
    
    print(f"🔍 총 {len(seg_files)}명의 환자 마스크 처리를 시작합니다.")

    for seg_file in tqdm(seg_files):
        patient_id = seg_file.replace('.nii.gz', '').replace('.nii', '')
        patient_img_dir = os.path.join(img_dir, patient_id)
        
        # DICOM 폴더가 없으면 건너뜀
        if not os.path.exists(patient_img_dir): 
            print(f"⚠️ {patient_id}: DICOM 폴더를 찾을 수 없어 건너뜁니다.")
            continue
        
        # [A] DICOM 물리적 정렬 (Z축 기준 내림차순: 머리 -> 가슴 순서)
        dcm_files = [f for f in os.listdir(patient_img_dir) if f.endswith('.dcm')]
        dcm_list = []
        for f in dcm_files:
            try:
                ds = pydicom.dcmread(os.path.join(patient_img_dir, f), stop_before_pixels=True)
                dcm_list.append({'z': ds.ImagePositionPatient[2], 'file': f})
            except Exception as e:
                print(f"❌ {f} 읽기 실패: {e}")
        
        # Z값이 큰 것(머리)부터 작은 것(다리) 순서로 정렬 (중요!)
        dcm_list.sort(key=lambda x: x['z'], reverse=True)
        
        # [B] NIfTI 마스크 로드
        seg_path = os.path.join(seg_dir, seg_file)
        mask_vol = nib.load(seg_path).get_fdata() 
        
        # [C] 환자별 저장 폴더 생성 (processed_masks/환자ID/)
        patient_save_dir = os.path.join(output_dir, patient_id)
        os.makedirs(patient_save_dir, exist_ok=True)
        
        # [D] 슬라이스별 저장
        # DICOM 개수와 NIfTI 슬라이스 개수가 다를 수 있으므로 최소값 기준 처리
        num_slices = min(mask_vol.shape[2], len(dcm_list))
        
        for s in range(num_slices):
            # 💡 핵심 로직: 
            # 1. s번째 정렬된 DICOM에 대해 NIfTI의 뒤에서부터(-s-1) 매칭 (역순 정렬 대응)
            slice_mask = mask_vol[:, :, -s-1] 
            
            # 2. 90도 회전하여 DICOM 이미지와 방향 일치 (사용자 정의 로직)
            slice_mask = np.rot90(slice_mask, k=1) 
            
            # 3. 정렬된 DICOM 파일명과 매칭해서 .npy 저장
            dcm_filename = dcm_list[s]['file']
            save_filename = dcm_filename.replace('.dcm', '.npy')
            
            np.save(os.path.join(patient_save_dir, save_filename), slice_mask.astype(np.uint8))

    print(f"\n✅ 정렬 및 분할 완료! 모든 마스크가 {output_dir}에 환자별로 저장되었습니다.")

# 실행
if __name__ == "__main__":
    process_and_align_fixed(SEG_DIR, IMAGE_DIR, PROC_DIR)

✅ 폴더 생성됨: C:\conda\3. Project\rsna-2022-cervical-spine-fracture-detection\old_train\processed_masks
🔍 총 87명의 환자 마스크 처리를 시작합니다.


100%|██████████| 87/87 [03:12<00:00,  2.21s/it]


✅ 정렬 및 분할 완료! 모든 마스크가 C:\conda\3. Project\rsna-2022-cervical-spine-fracture-detection\old_train\processed_masks에 환자별로 저장되었습니다.


## dicom, nifti 방향 전수 조사

In [11]:
if not os.path.exists(REPORT_DIR):
    os.makedirs(REPORT_DIR)

def generate_alignment_reports():
    # 처리된 마스크가 있는 환자 목록 가져오기
    patients = [d for d in os.listdir(PROC_DIR) if os.path.isdir(os.path.join(PROC_DIR, d))]
    print(f"🔎 총 {len(patients)}명의 환자에 대한 전수조사를 시작합니다.")

    for patient_id in tqdm(patients):
        patient_mask_dir = os.path.join(PROC_DIR, patient_id)
        npy_files = [f for f in os.listdir(patient_mask_dir) if f.endswith('.npy')]
        
        if not npy_files: continue

        # 🎯 전략: 마스크가 있는 중간 슬라이스 찾기
        # 전체 슬라이스 중 중간 지점 근처의 마스크를 선택하여 시각화 효과 극대화
        mid_idx = len(npy_files) // 2
        sample_npy = npy_files[mid_idx]
        slice_num = sample_npy.replace('.npy', '')

        # 경로 설정
        dcm_path = os.path.join(IMAGE_DIR, patient_id, f"{slice_num}.dcm")
        npy_path = os.path.join(patient_mask_dir, sample_npy)

        if not os.path.exists(dcm_path): continue

        # --- 데이터 로드 및 전처리 (사용자 로직 반영) ---
        ds = pydicom.dcmread(dcm_path)
        img = ds.pixel_array
        
        # HU 변환 및 Bone Windowing
        slope = float(ds.RescaleSlope) if 'RescaleSlope' in ds else 1.0
        intercept = float(ds.RescaleIntercept) if 'RescaleIntercept' in ds else 0.0
        img = img * slope + intercept
        img = np.clip(img, 350 - 1350, 350 + 1350)
        
        mask = np.load(npy_path)

        # --- 시각화 생성 ---
        fig, axes = plt.subplots(1, 3, figsize=(18, 6))
        fig.suptitle(f"Patient: {patient_id} | Slice: {slice_num}", fontsize=15)

        axes[0].imshow(img, cmap='gray')
        axes[0].set_title("DICOM")
        axes[0].axis('off')

        axes[1].imshow(mask, cmap='nipy_spectral') 
        axes[1].set_title("Mask (.npy)")
        axes[1].axis('off')

        axes[2].imshow(img, cmap='gray')
        masked_data = np.ma.masked_where(mask == 0, mask)
        axes[2].imshow(masked_data, cmap='autumn', alpha=0.5) 
        axes[2].set_title("Overlay Check")
        axes[2].axis('off')

        # --- 파일로 저장 (plt.show 대신 저장!) ---
        save_path = os.path.join(REPORT_DIR, f"{patient_id}_check.png")
        plt.tight_layout()
        plt.savefig(save_path)
        plt.close(fig) # 메모리 해제

    print(f"\n✅ 전수조사 완료! '{REPORT_DIR}' 폴더에서 이미지를 확인하세요.")

# 실행
if __name__ == "__main__":
    generate_alignment_reports()

🔎 총 87명의 환자에 대한 전수조사를 시작합니다.


100%|██████████| 87/87 [00:30<00:00,  2.81it/s]


✅ 전수조사 완료! 'C:\conda\3. Project\rsna-2022-cervical-spine-fracture-detection\old_train\alignment_reports' 폴더에서 이미지를 확인하세요.


## 데이터 분류

In [14]:
def check_label_overlap():
    # A. Segmentation 마스크가 있는 환자 ID 추출
    if not os.path.exists(SEG_DIR):
        print("⚠️ 마스크 폴더가 없습니다.")
        return []
        
    seg_files = [f for f in os.listdir(SEG_DIR) if f.endswith('.nii') or f.endswith('.nii.gz')]
    seg_patient_ids = set([f.split('.nii')[0] for f in seg_files])
    
    # B. Bounding Box 정보가 있는 환자 ID 추출
    if not os.path.exists(BBOX_CSV):
        print("⚠️ BBOX CSV 파일이 없습니다.")
        return list(seg_patient_ids)
        
    bbox_df = pd.read_csv(BBOX_CSV)
    bbox_patient_ids = set(bbox_df['StudyInstanceUID'].unique())
    
    # C. 교집합 및 차집합 계산
    overlap = seg_patient_ids.intersection(bbox_patient_ids)
    only_seg = seg_patient_ids - bbox_patient_ids
    only_bbox = bbox_patient_ids - seg_patient_ids
    
    print(f"📊 --- 데이터 라벨 중복 분석 결과 ---")
    print(f"✅ Mask와 Box 모두 있음 (Golden): {len(overlap)}명")
    print(f"💠 Mask만 있음 (Seg Only): {len(only_seg)}명")
    print(f"📦 Box만 있음 (BBox Only): {len(only_bbox)}명")
    print(f"--------------------------------------")
    
    return list(overlap)

# 실행
overlap_patients = check_label_overlap()

📊 --- 데이터 라벨 중복 분석 결과 ---
✅ Mask와 Box 모두 있음 (Golden): 40명
💠 Mask만 있음 (Seg Only): 47명
📦 Box만 있음 (BBox Only): 195명
--------------------------------------


## Golden dataset 생성

In [22]:
def create_tiered_dataset_v4():
    # A. 환자 그룹 분류 로직
    seg_files = [f for f in os.listdir(SEG_DIR) if f.endswith('.nii') or f.endswith('.nii.gz')]
    seg_ids = set([f.split('.nii')[0] for f in seg_files])
    
    bbox_df = pd.read_csv(BBOX_CSV)
    bbox_ids = set(bbox_df['StudyInstanceUID'].unique())
    
    groups = {
        '1_mask_and_box': seg_ids.intersection(bbox_ids),
        '2_only_mask': seg_ids - bbox_ids,
        '3_only_box': bbox_ids - seg_ids
    }

    # B. 데이터 복사 및 정리
    for g_name, p_ids in groups.items():
        if not p_ids: continue
        print(f"\n📂 {g_name} 그룹 분류 중... (대상: {len(p_ids)}명)")
        
        for pid in tqdm(p_ids):
            # 1. 원본 DICOM 이미지 (train_images)
            src_img = os.path.join(IMAGE_DIR, pid)
            if os.path.exists(src_img):
                target = os.path.join(GOLDEN_DATA_DIR, g_name, 'train_images', pid)
                shutil.copytree(src_img, target, dirs_exist_ok=True)

            # 2. NIfTI 마스크 (segmentations)
            if pid in seg_ids:
                for ext in ['.nii.gz', '.nii']:
                    src_seg = os.path.join(SEG_DIR, pid + ext)
                    if os.path.exists(src_seg):
                        target_dir = os.path.join(GOLDEN_DATA_DIR, g_name, 'segmentations')
                        os.makedirs(target_dir, exist_ok=True)
                        shutil.copy2(src_seg, target_dir)

            # 3. 슬라이스된 .npy 마스크 (processed_masks)
            src_proc = os.path.join(PROC_DIR, pid)
            if os.path.exists(src_proc):
                target = os.path.join(GOLDEN_DATA_DIR, g_name, 'processed_masks', pid)
                shutil.copytree(src_proc, target, dirs_exist_ok=True)

            # 4. 환자별 요약 CSV (patient_summaries)
            if os.path.exists(SUMMARY_DIR):
                summary_files = [f for f in os.listdir(SUMMARY_DIR) if f.startswith(pid)]
                for sf in summary_files:
                    target_dir = os.path.join(GOLDEN_DATA_DIR, g_name, 'patient_summaries')
                    os.makedirs(target_dir, exist_ok=True)
                    shutil.copy2(os.path.join(SUMMARY_DIR, sf), target_dir)

            # 5. 변환된 전체 NIfTI 이미지 (converted_images)
            # 파일 형태(pid_image.nii.gz)일 수도 있고 폴더(pid/)일 수도 있음
            # 먼저 파일 형태를 체크
            conv_file_name = f"{pid}_image.nii.gz"
            src_conv_file = os.path.join(CONV_DIR, conv_file_name)
            src_conv_dir = os.path.join(CONV_DIR, pid)

            if os.path.exists(src_conv_file): # 파일인 경우
                target_dir = os.path.join(GOLDEN_DATA_DIR, g_name, 'converted_images')
                os.makedirs(target_dir, exist_ok=True)
                shutil.copy2(src_conv_file, target_dir)
            elif os.path.exists(src_conv_dir): # 폴더인 경우
                target = os.path.join(GOLDEN_DATA_DIR, g_name, 'converted_images', pid)
                shutil.copytree(src_conv_dir, target, dirs_exist_ok=True)

    print(f"\n✨ 모든 그룹 데이터 정리가 완료되었습니다! 경로: {GOLDEN_DATA_DIR}")

# 실행
create_tiered_dataset_v4()


📂 1_mask_and_box 그룹 분류 중... (대상: 40명)


100%|██████████| 40/40 [01:15<00:00,  1.89s/it]



📂 2_only_mask 그룹 분류 중... (대상: 47명)


100%|██████████| 47/47 [03:59<00:00,  5.10s/it]



📂 3_only_box 그룹 분류 중... (대상: 195명)


100%|██████████| 195/195 [04:22<00:00,  1.35s/it]


✨ 모든 그룹 데이터 정리가 완료되었습니다! 경로: C:\conda\3. Project\rsna-2022-cervical-spine-fracture-detection\golden_dataset


## csv 파일 변경

In [4]:
def save_filtered_csv():
    # 데이터 로드
    df_train = pd.read_csv(TRAIN_CSV)
    df_bbox = pd.read_csv(BBOX_CSV)

    # A. 대상 환자 ID 리스트 추출
    # 1. Segmentation 파일이 있는 환자 ID (87명)
    seg_files = [f for f in os.listdir(SEG_DIR) if f.endswith('.nii') or f.endswith('.nii.gz')]
    seg_ids = set([f.split('.nii')[0] for f in seg_files])

    # 2. Bounding Box 정보가 있는 환자 ID (235명)
    bbox_ids = set(df_bbox['StudyInstanceUID'].unique())

    # 3. 전체 대상 환자 합집합 (중복 제외 총 인원)
    target_ids = list(seg_ids.union(bbox_ids))
    print(f"📊 필터링 대상 환자 수: {len(target_ids)}명")

    # B. 데이터 필터링
    # 1. train.csv 필터링
    filtered_train = df_train[df_train['StudyInstanceUID'].isin(target_ids)].reset_index(drop=True)
    
    # 2. train_bounding_boxes.csv 필터링
    filtered_bbox = df_bbox[df_bbox['StudyInstanceUID'].isin(target_ids)].reset_index(drop=True)

    # C. 파일 저장 (골든 데이터셋 폴더 내로 이동)
    filtered_train.to_csv(FILTERED_TRAIN_CSV, index=False)
    filtered_bbox.to_csv(FILTERED_BBOX_CSV, index=False)

    print(f"\n✨ 필터링 완료 및 골든 데이터셋 이동 완료!")
    print(f"📁 저장된 위치: {GOLDEN_DATA_DIR}")
    print(f" - {os.path.basename(FILTERED_TRAIN_CSV)} (행 수: {len(filtered_train)})")
    print(f" - {os.path.basename(FILTERED_BBOX_CSV)} (행 수: {len(filtered_bbox)})")

if __name__ == "__main__":
    save_filtered_csv()

📊 필터링 대상 환자 수: 282명

✨ 필터링 완료 및 골든 데이터셋 이동 완료!
📁 저장된 위치: C:\conda\3. Project\rsna-2022-cervical-spine-fracture-detection\golden_dataset
 - filtered_train.csv (행 수: 282)
 - filtered_train_bounding_boxes.csv (행 수: 7217)
